In [10]:
from nltk.stem.wordnet import WordNetLemmatizer
import re
import spacy
import os
import pandas as pd
import numpy as np
import datetime 
nlp = spacy.load('en_core_web_sm', disable=['ner', 'textcat'])


In [2]:

# def extract_cities(Dataframe,df):
#     df["Areas"].str.lower()
#     Data_of_region =  df["Areas"].values.tolist()
#     Data_of_region = [each_city.lower() for each_city in Data_of_region]
#     indexs = []
#     notindexes = []
#     for i in range(len(Dataframe)):
#         doc = nlp(Dataframe[i][0])
        
#         city_found = False
#         for city in doc.ents:
#             city = str(city).lower()
#             if city in Data_of_region:
#                 indexs.append(i)
#                 city_found = True
#         if city_found == False:
#             notindexes.append(i)
            
#     return notindexes,indexs 

In [8]:
todays_date = datetime.date.today()
path = str(todays_date.year)
General_CAT = ["front-page", "back-page",
               "national", "business", "international", "sport"]
Metro_CAT = ["karachi", "lahore", "islamabad", "peshawar"]
files = []
Previous_Date = todays_date - datetime.timedelta(days=1)
Previous_Date = Previous_Date.strftime('%Y-%m-%d')
for i in range(len(General_CAT)):
    files.append(path+"/"+str(Previous_Date)+"/"+General_CAT[i]+".csv")
for i in range(len(Metro_CAT)):
    files.append(path+"/"+str(Previous_Date)+"/"+Metro_CAT[i]+".csv")

df = pd.read_csv(files[0], header=0)
df = df.iloc[:, 1:]
for i in range(len(files)-1):
    read_ = pd.read_csv(files[i+1],header=0)
    read_ = read_.iloc[:, 1:]
    df = pd.concat([df, read_])
Dataframe = df.values.tolist()


In [16]:
def clean(doc):
    doc = doc.lower()
    doc = nlp(doc)
    tokens = [tokens.lower_ for tokens in doc]
    tokens = [tokens for tokens in doc if (tokens.is_stop == False)]
    tokens = [tokens for tokens in tokens if (tokens.is_punct == False)]
    final_token = [WordNetLemmatizer().lemmatize(token.text)
                   for token in tokens]

    return " ".join(final_token)


In [11]:
# split sentences
def sentences(text):
    # split sentences and questions
    text = re.split('[.?]', text)
    clean_sent = []
    for sent in text:
        clean_sent.append(sent)
    return clean_sent


In [12]:
def getlocations():
    # Loading dataset 
    df = pd.read_csv('Alldata_refined.csv')
    # Droping NULL rows
    df = df.dropna()
    # Extracting location col
    df = df["Locations"]
    # Converting Data frame to sorted list in lower case
    Data_of_region = df.values.tolist()
    Data_of_region = [each_city.lower() for each_city in Data_of_region]
    Data_of_region = list(dict.fromkeys(sorted(Data_of_region)))
    # Storing indexes of each alphabet starting index 
    index = dict()
    # Helping variables to store indexes 
    flag= False
    push = False
    current_alphabet = ""
    start = 0
    finish = 0
    # Creating index hash 
    for i in range(len(Data_of_region)):
        if i != 0 and Data_of_region[i][0] != Data_of_region[i][0]:
            flag = True
            push = True
            finish = i-1
        if push == True:
            index[current_alphabet] = finish
        if flag == False:
            start = i
            current_alphabet = Data_of_region[i][0]
            index.__setitem__(current_alphabet, i)
    return index,Data_of_region

In [29]:
def Get_location(read_more,header):
    index, Data_of_region = getlocations()
    header = header.lower()
    header = clean(header)
    header = header.split()
    text = sentences(clean(read_more))
    cities = dict()
    for i in text:
        doc = nlp(i)
        for token in range(len(doc)):
                if doc[token].pos_ == "PROPN":
                    flag = False
                    end = index[doc[token].text.lower()[0]]
                    if end == index['a']:
                        start = 0
                    else:
                        start = chr(ord(doc[token].text.lower()[0])-1)
                        while(True):
                            if start in index: 
                                start = index[start]-1
                                break
                            else:
                                start = chr(ord(start)-1)
                    area_count = 0
                    for areas in range(start,end):
                        words = Data_of_region[areas].split()
                        subtoken = token
                        checker = []
                        for iterator in range(len(words)):
                            if subtoken + iterator < len(doc):
                                if doc[subtoken+iterator].text.lower() == words[iterator]:
                                    checker.append(1)
                        if len(checker) == len(words):
                            city = ' '.join(words)
                            area_count = len(words[iterator])
                            flag = True
                            break
                    if flag:
                        match = False
                        word1 = ""
                        word2 = ""
                        if token != 0:
                            word1 = doc[token-1].text.lower()
                        subtoken = token + area_count
                        if subtoken + 1 < len(doc):
                            word2 = doc[subtoken+1].text.lower()
                        if word1 in header or word2 in header:
                            match = True
                        if city in cities:
                                if match:
                                    cities[city] += 3
                                else:
                                    cities[city] += 1
                        else:
                            if match:
                                cities.__setitem__(city, 3)
                            else:
                                cities.__setitem__(city, 1)
    return cities


In [34]:
# header = "High-level flood in Sukkur Barrage as casualties rise by 57"
# header="123"
# print(Dataframe[0][0])
# print(Dataframe[0][2])
print(Get_location(Dataframe[10][2], Dataframe[10][0]))


{'chamber': 1, 'balochistan': 1, 'khyber pakhtunkhwa': 1}


In [1]:
print(Dataframe[11][0])
print(Dataframe[11][2])

NameError: name 'Dataframe' is not defined

In [1]:
from unittest import result
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
import re
import pandas as pd
import pandas as pd
import pathlib
import json
import readline
from sutime import SUTime
from datetime import datetime
nlp = spacy.load('en_core_web_sm', disable=['ner', 'textcat'])


class parser():
    def __init__(self):
        pass

    def clean(self, doc):
        doc = doc.lower()
        doc = nlp(doc)
        tokens = [tokens.lower_ for tokens in doc]
        tokens = [tokens for tokens in doc if (tokens.is_stop == False)]
        tokens = [tokens for tokens in tokens if (tokens.is_punct == False)]
        final_token = [WordNetLemmatizer().lemmatize(token.text)
                       for token in tokens]

        return " ".join(final_token)

    # split sentences

    def sentences(self,text):
        # split sentences and questions
        text = re.split('[.?]', text)
        clean_sent = []
        for sent in text:
            clean_sent.append(sent)
        return clean_sent

    def load_cities(self, file):
        # Loading dataset
        df = pd.read_csv(file)
        # Droping NULL rows
        df = df.dropna()
        # Extracting location col
        df = df["Locations"]
        # Converting Data frame to sorted list in lower case
        Data_of_region = df.values.tolist()
        Data_of_region = [each_city.lower() for each_city in Data_of_region]
        Data_of_region = list(dict.fromkeys(sorted(Data_of_region)))
        # Storing indexes of each alphabet starting index
        index = dict()
        # Helping variables to store indexes
        flag = False
        push = False
        current_alphabet = ""
        start = 0
        finish = 0
        # Creating index hash
        for i in range(len(Data_of_region)):
            if i != 0 and Data_of_region[i][0] != Data_of_region[i][0]:
                flag = True
                push = True
                finish = i-1
            if push == True:
                index[current_alphabet] = finish
            if flag == False:
                start = i
                current_alphabet = Data_of_region[i][0]
                index.__setitem__(current_alphabet, start)
        self.index = index
        self.Data_of_region = Data_of_region

    def Get_location(self, read_more, header):
        self.load_cities(r'Alldata_refined.csv')
        header = header.lower()
        header = self.clean(header)
        header = header.split()
        text = self.sentences(self.clean(read_more))
        cities = dict()
        for i in text:
            doc = nlp(i)
            for token in range(len(doc)):
                if doc[token].pos_ == "PROPN":
                    flag = False
                    end = self.index[doc[token].text.lower()[0]]
                    if end == self.index['a']:
                        start = 0
                    else:
                        start = chr(ord(doc[token].text.lower()[0])-1)
                        while (True):
                            if start in self.index:
                                start = self.index[start]-1
                                break
                            else:
                                start = chr(ord(start)-1)
                    area_count = 0
                    for areas in range(start, end):
                        words = self.Data_of_region[areas].split()
                        subtoken = token
                        checker = []
                        for iterator in range(len(words)):
                            if subtoken + iterator < len(doc):
                                if doc[subtoken+iterator].text.lower() == words[iterator]:
                                    checker.append(1)
                        if len(checker) == len(words):
                            city = ' '.join(words)
                            area_count = len(words[iterator])
                            flag = True
                            break
                    if flag:
                        match = False
                        word1 = ""
                        word2 = ""
                        if token != 0:
                            word1 = doc[token-1].text.lower()
                        subtoken = token + area_count
                        if subtoken + 1 < len(doc):
                            word2 = doc[subtoken+1].text.lower()
                        if word1 in header or word2 in header:
                            match = True
                        if city in cities:
                            if match:
                                cities[city] += 3
                            else:
                                cities[city] += 1
                        else:
                            if match:
                                cities.__setitem__(city, 3)
                            else:
                                cities.__setitem__(city, 1)
        return cities

    def checkDate(self, parsedData, referenceDate):
        flag = False
        for tag in parsedData:
            if tag['type'] == 'DATE':
                # if datetime.strptime(tag['value'], '%Y-%m-%d') <= datetime.strptime(referenceDate, '%Y-%m-%d'):
                try:
                    if datetime.strptime(tag['value'], '%Y-%m-%d') <= datetime.strptime(referenceDate, '%Y-%m-%d'):
                        flag = True
                except:
                    pass
        return flag

    def extractDate(self, parsedData, referenceDate):
        focusTime = ''
        for tag in parsedData:
            if tag['type'] == 'DATE':
                try:
                    if datetime.strptime(tag['value'], '%Y-%m-%d') <= datetime.strptime(referenceDate, '%Y-%m-%d'):
                        focusTime = tag['value']
                        return focusTime
                except:
                    pass   


    def Get_time(self, document):
        sutime = SUTime()
        # print(json.dumps(sutime.parse(test_case), sort_keys=True, indent=4))
        timeData = dict()
        # print(str)
        count = 0
        for data in document.itertuples():
            headerParse = sutime.parse(data[2], reference_date=data[5])
            self.checkDate(headerParse, data[5])
            if not headerParse or not self.checkDate(headerParse, data[5]):
                summaryParse = sutime.parse(data[3], reference_date=data[5])
                self.checkDate(summaryParse, data[5])
                if not summaryParse or not self.checkDate(summaryParse, data[5]):
                    details = data[4]
                    lines = details.split('\n')
                    del lines[-2]
                    details = '\n'.join(lines)
                    detailsParse = sutime.parse(details, reference_date=data[5])
                    self.checkDate(detailsParse, data[5])
                    if not detailsParse:
                        count += 1
                    else:
                        timeData[data[0]] = dict()
                        timeData[data[0]]["Creation_Date"] = data[5]
                        timeData[data[0]]["Header"] = dict()
                        timeData[data[0]]["Header"]["Text"] = data[2]
                        timeData[data[0]]['Summary'] = dict()
                        timeData[data[0]]['Summary']["Text"] = data[3]
                        timeData[data[0]]['Details'] = dict()
                        timeData[data[0]]['Details']["Text"] = data[4]
                        timeData[data[0]]['Details']["Tags"] = detailsParse
                        timeData[data[0]]["focusTime"] = self.extractDate(detailsParse, data[5])
                else:
                    timeData[data[0]] = dict()
                    timeData[data[0]]["Creation_Date"] = data[5]
                    timeData[data[0]]["Header"] = dict()
                    timeData[data[0]]["Header"]["Text"] = data[2]
                    timeData[data[0]]['Summary'] = dict()
                    timeData[data[0]]['Summary']["Text"] = data[3]
                    timeData[data[0]]['Details'] = dict()
                    timeData[data[0]]['Details']["Text"] = data[4]
                    timeData[data[0]]['Summary']["Tags"] = summaryParse
                    timeData[data[0]]["focusTime"] = self.extractDate(summaryParse, data[5])
            else:
                timeData[data[0]] = dict()
                timeData[data[0]]["Creation_Date"] = data[5]
                timeData[data[0]]["Header"] = dict()
                timeData[data[0]]["Header"]["Text"] = data[2]
                timeData[data[0]]['Summary'] = dict()
                timeData[data[0]]['Summary']["Text"] = data[3]
                timeData[data[0]]['Details'] = dict()
                timeData[data[0]]['Details']["Text"] = data[4]
                timeData[data[0]]["Header"]["Tags"] = headerParse
                timeData[data[0]]["focusTime"] = self.extractDate(headerParse, data[5])
            if timeData[data[0]]["focusTime"] == 'null':
                timeData[data[0]]["focusTime"] = data[5]
        return timeData

def main():
    filename = r"C:\Danyal\Work\FAST\Semester 7\Final Year Project - I\Dummy Project\NAaaS\Data_Generator\2022\2022-09-12\front-page.csv"
    path = pathlib.PurePath(filename)
    df = pd.read_csv(filename, index_col=None, header=0, dtype="string")
    df['Creation_Date'] = path.parent.name
    Parser = parser()
    results = Parser.Get_time(df)

    for data in df.itertuples():
        result = Parser.Get_location(data[4], data[2])
        results[data[0]]["Locations"] = result
    print(json.dumps(results, indent=4))

main()    
    


{
    "0": {
        "Creation_Date": "2022-09-12",
        "Header": {
            "Text": "Flood response goes digital to ensure transparency"
        },
        "Summary": {
            "Text": "The dashboard will provide direct information to the public about the financial support, relief goods being received and distributed."
        },
        "Details": {
            "Text": "ISLAMABAD: In a bid to ensure transparency in the allocation of relief funds for the flood survivors, the federal government has decided to establish a \u2018Digital Flood Dashboard\u2019, which will keep the public abreast of relief measures taken by the authorities concerned.\nThe dashboard prepared with the help of latest technology will be launched by Planning Minister and National Flood Response and Coordination Centre chief Ahsan Iqbal on Monday (today) following the directives of Prime Minister Shehbaz Sharif.\nIt will provide direct information to the general public about the financial support and t